In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
DATA_PATH = 'data/'

In [3]:
df_tweets = pd.read_csv(DATA_PATH+'tweets.csv')

KeyboardInterrupt: 

In [4]:
df_users = pd.read_csv(DATA_PATH+'users.csv')

In [5]:
df_users.head()

,id,name,lang,bot,created_at,statuses_count
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54.0
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3.0
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50.0
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085.0


In [6]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11508 non-null  int64  
 1   name            11507 non-null  object 
 2   lang            11508 non-null  object 
 3   bot             11508 non-null  int64  
 4   created_at      11508 non-null  object 
 5   statuses_count  11109 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 539.6+ KB


Check if any user record has a null value in it
Extract only record with a null name (id = 1535)

In [7]:
df_null = df_users.isnull()
idx_null = df_users.index[df_null["name"] == True].tolist()
df_users.iloc[idx_null]

,id,name,lang,bot,created_at,statuses_count
1535,2166124159,NaN,en,0,2018-11-02 06:39:14,6566.0


In [9]:
df_users.duplicated(['id']).value_counts()

False    11508
dtype: int64

Print info about tweets file

In [ ]:
df_tweets.head()

In [ ]:
df_tweets.info(null_counts=True)

In [ ]:
df_tweets.size

## Verifica esistenza record duplicati

In [ ]:
df_tweets_dupl = df_tweets.duplicated()
df_tweets_dupl.value_counts()

## Drop duplicates

In [ ]:
df_tweets_clear = df_tweets.drop_duplicates(keep = 'first')
df_tweets_clear.shape[0]

In [ ]:
df_tweets_clear.info(show_counts=True)

## Verifica di duplicati su un sottoinsieme di colonne

In [ ]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id']) # Stesso id
df_tweets_dupl.value_counts()

In [ ]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id', 'user_id']) # Stesso id e stesso user_id
df_tweets_dupl.value_counts()

In [ ]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id', 'user_id', 'text']) # Stessi id, user_id e testo
df_tweets_dupl.value_counts()

In [ ]:
df_tweets_dupl = df_tweets_clear.duplicated(subset=['id', 'user_id', 'created_at', 'text']) # Stessi id, user_id, testo e data di creazione
df_tweets_dupl.value_counts()

In [ ]:
df_tweets_dupl =df_tweets_clear[df_tweets_clear.duplicated(subset=['id', 'user_id','text'])]

## Data la presenza di molteplici valori nulli tra i duplicati secondo le colonne [id, user_id, text] verifichiamo la corrispondenza di tweet con lo user_id non nullo con la tabella users

In [ ]:
df_nonnull_users = df_tweets_dupl[df_tweets_dupl['user_id'].isnull() == False] # Solo 12 user_id sono avvalorati nei duplicati
df_nonnull_users.info()

In [ ]:
df_nonnull_users.user_id = pd.to_numeric(df_nonnull_users.user_id) # transform column to int values from object
df_nonnull_users.merge(df_users, left_on='user_id', right_on='id') # merge with users table

All users have a correspondance with users in the users file 

## Eliminazione delle righe ripetute secondo il pattern sopracitato aventi lo user_id nullo

In [ ]:
df_tweets_more_clear = df_tweets_clear[df_tweets_clear.duplicated(subset=['id', 'user_id','text'])]

In [ ]:
df_tweets_more_clear.info()

Check null user_id values

In [ ]:
df_tweets_more_clear[df_tweets_more_clear['user_id'].isnull() == True]

Drop all records where user_id is null

In [ ]:
df_tweets_single =df_tweets_clear.drop(df_tweets_more_clear[df_tweets_more_clear['user_id'].isnull() == True].index)
df_tweets_single.info(show_counts=True)

Take all users in duplicates that have a user_id that is not null

In [ ]:
df_nonnull_users = df_tweets_dupl[df_tweets_dupl['user_id'].isnull() == False]
df_nonnull_users.info()

In [ ]:
df_nonnull_users.user_id = pd.to_numeric(df_nonnull_users.user_id) # transform column to int values from object
df_nonnull_users.merge(df_users, left_on='user_id', right_on='id') # merge with users table

In [ ]:
for attr in ['retweet_count','reply_count','favorite_count','num_hashtags','num_urls','num_mentions']:
    cazzo = df_tweets_single[df_tweets_single.duplicated(subset=['id', 'user_id', 'text', attr])]
    print(attr+'\n'+str(cazzo.shape[0])+'\n')
    print(cazzo)

As this print shows, most of duplicates have missing values. If we exclude `retweet_counts`, the duplicates are the same.

In [ ]:
df_tweets_single[df_tweets_single.duplicated(subset=['id', 'user_id', 'text', 'reply_count','favorite_count','num_hashtags','num_urls','num_mentions'], keep=False)].sort_values(by='id')

In [ ]:
df_tweets_single.to_csv(sep = '#', path_or_buf = DATA_PATH+'tweets_no_dupl.csv', index = False)

From now on we assume that the records in the dataset are not duplcated.